# Project: Data Modeling with AWS Redshift
## Data Analytics 

#### Step 1 - Importing packages

In [1]:
# General:
import pandas as pd
import pandas.io.sql as sqlio
import numpy as np
import os
import re


# Database:
import psycopg2
import keyring
import getpass

# My libraries:
import sys
sys.path.insert(0, '../src/scripts')

# Warnings
import warnings; warnings.simplefilter('ignore')

#### Step 2 - Configuration File

In [2]:
import configparser

config = configparser.ConfigParser()
config.read('../scripts/dwh.cfg')

['../scripts/dwh.cfg']

In [3]:
config

#### Step 3 - Define Connection to AWS Redshift Database

In [4]:
'''
Database Connection

Requirements: 
    1. AWS Redshift credentials
'''

p = keyring.get_password('redshift', 'user_name')
HOST = config.get('CLUSTER', 'HOST')
DB_NAME =  config.get('CLUSTER', 'DB_NAME')
DB_USER = config.get('CLUSTER', 'DB_USER')
DB_PASSWORD = config.get('CLUSTER', 'DB_PASSWORD')
DB_PORT = config.get('CLUSTER', 'DB_PORT')

a = "user='{}' password='{}' host='{}' port='{}' dbname='{}' ".format(DB_USER, DB_PASSWORD, HOST, DB_PORT, DB_NAME)

#print(a)

try:
    conn = psycopg2.connect(
        "user='{}' password='{}' host='{}' port='{}' dbname='{}' ".format(DB_USER, DB_PASSWORD, HOST, DB_PORT, DB_NAME))

    cursor = conn.cursor()
    print('I am connected to the database!')
except:
    print('I am unable to connect to the database! Please, enter with your credentials.')

I am connected to the database!


#### Step 4 - Extracting data

#### Check tables

In [5]:
# Queries on redshift
QS =  """select count(1) as ct, 'staging_events' as table
        from staging_events
        union
        select count(1) as ct, 'staging_songs' as table
        from staging_songs
        union
        select count(1) as ct, 'artists' as table
        from artists
        union
        select count(1) as ct, 'songs' as table
        from songs
        union
        select count(1) as ct, 'time' as table
        from time
        union
        select count(1) as ct, 'users' as table
        from users
        union
        select count(1) as ct, 'songplays' as table
        from songplays
  """

q1 = sqlio.read_sql_query(QS, conn);
q1.head(12)

,ct,table
0,14896,staging_songs
1,8056,staging_events
2,6813,time
3,10025,artists
4,9957,songplays
5,104,users
6,14896,songs


#### Analysis

In [6]:
# Queries on redshift
sql_query = """ 
        SELECT sp.songplay_id,
           u.user_id,
           s.song_id,
           u.last_name,
           sp.start_time,
           a.name,
           s.title
        FROM songplays AS sp
                 JOIN users AS u
                      ON u.user_id = sp.user_id
                 JOIN songs AS s
                      ON s.song_id = sp.song_id
                 JOIN artists AS a
                      ON a.artist_id = sp.artist_id
                 JOIN time AS t
                      ON t.start_time = sp.start_time
        ORDER BY sp.start_time
        LIMIT 1000;
        """

In [7]:
q1 = sqlio.read_sql_query(sql_query, conn);

#### Step 5 - Data analysis and validation

In [8]:
q1.shape

(1000, 7)

In [9]:
q1.head(5)

,songplay_id,user_id,song_id,last_name,start_time,name,title
0,11126,8,SOKOZQI12A8C13C538,Summers,2018-11-01 21:05:52,Mr Oizo,Shortkut
1,18806,8,SOQVXCF12AF72A419B,Summers,2018-11-01 21:11:13,The Mars Volta,Take The Veil Cerpin Taxt
2,16945,8,SOGAALD12A6D4F58F2,Summers,2018-11-01 21:11:13,The Mars Volta,Caviglia
3,13979,8,SOEIQUY12AF72A086A,Summers,2018-11-01 21:11:13,The Mars Volta,Eriatarka
4,18324,8,SOICPZO12A8C13F712,Summers,2018-11-01 21:17:33,Infected Mushroom,In Front Of Me


In [10]:
# Queries on redshift
q2 = """ 
        SELECT COUNT(*) FROM songplays;
        """
q2 = sqlio.read_sql_query(q2, conn);

q2.head(5)

,count
0,9957


In [11]:
# Queries on redshift
q3 =  """
    SELECT (u.first_name + ' ' + u.last_name) as user,
            COUNT(s.title) as listened
    FROM songplays as sp
    JOIN songs as s
        ON sp.song_id = s.song_id
    JOIN artists as a
        ON sp.artist_id = a.artist_id
    JOIN users as u
        ON sp.user_id = u.user_id
    GROUP BY u.first_name, u.last_name
    ORDER BY 2 DESC
"""
q3 = sqlio.read_sql_query(q3, conn);

q3.head(5)

,user,listened
0,Chloe Cuevas,2242
1,Tegan Levine,2178
2,Lily Koch,1466
3,Jacqueline Lynch,1112
4,Mohammad Rodriguez,894


In [12]:
# Queries on redshift
q4 =  """SELECT (u.first_name + ' ' + u.last_name) as user,
       s.title                            as song,
       a.name                             as artist,
       t.day,
       t.month,
       t.year
    FROM songplays as sp
         JOIN songs as s
              ON sp.song_id = s.song_id
         JOIN artists as a
              ON sp.artist_id = a.artist_id
         JOIN users as u
              ON sp.user_id = u.user_id
         JOIN time as t
              ON sp.start_time = t.start_time
    where t.year = 2018
          and a.name = 'Depeche Mode'
  """

In [13]:
q4r = sqlio.read_sql_query(q4, conn);

q4r.head(6)

,user,song,artist,day,month,year
0,Layla Griffin,Precious (Motor Remix),Depeche Mode,19,11,2018
1,Mohammad Rodriguez,Precious (Motor Remix),Depeche Mode,26,11,2018
2,Mohammad Rodriguez,Precious (Motor Remix),Depeche Mode,26,11,2018
3,Chloe Cuevas,Precious (Motor Remix),Depeche Mode,15,11,2018
4,Chloe Cuevas,Precious (Motor Remix),Depeche Mode,15,11,2018
5,Layla Griffin,Peace (Justus Köhncke Extended Disco Club Vocal),Depeche Mode,19,11,2018


-----------